### dGen Post-Run Analysis (Baseline vs Policy)
 - Loads per-state CSVs (baseline/policy)
 - Computes portfolio & cumulative bill savings (cohort carry-forward via `new_adopters`)
 - Aggregates: batt_kwh_cum, system_kw_cum, number_of_adopters, medians
 - Faceted plots by state (Baseline vs Policy)
 - EIA price compare

In [ ]:
# Set path to access helpers
from pathlib import Path
import sys

PARENT = Path.cwd().parent     # helpers live here
if str(PARENT) not in sys.path:
    sys.path.insert(0, str(PARENT))

In [ ]:
# Imports
from analysis_functions import (
    SavingsConfig, process_all_states,
    facet_lines_by_state, bar_tech_potential_2040, facet_lines_national_totals,
    process_all_states_peaks, facet_peaks_by_state, facet_median_storage_by_state,
    facet_state_peak_timeseries_from_hourly, export_compiled_results_to_excel, build_national_totals,
    choropleth_pv_delta_gw_policy_vs_baseline, plot_us_cum_adopters_grouped, policy_only_bill_price_diffs_after_adoption
)
import importlib, analysis_functions as af  # for reloads after editing the file
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")

# --- Paths / settings ---
ROOT_DIR = "/Volumes/Seagate Portabl/permit_power/dgen_runs/per_state_outputs"
RUN_ID   = "run_all_states_add_finance_series"
N_JOBS   = 8
cfg      = SavingsConfig(lifetime_years=30, cap_to_horizon=False)

# If you just edited analysis_functions.py, reload it:
importlib.reload(af)



#### Process all states (parallel)
##### Produces small, tidy DataFrames ready for plotting:
 - result["totals"] → batt_kwh_cum, system_kw_cum, number_of_adopters
 - result["portfolio_annual_savings"], result["cumulative_bill_savings"]
 - result["median_system_kw"], result["tech_2040"], result["lifetime_totals"], ...



In [ ]:
result   = process_all_states(root_dir=ROOT_DIR, run_id=RUN_ID, n_jobs=1, cfg=cfg)
peaks_df = process_all_states_peaks(root_dir=ROOT_DIR, run_id=RUN_ID, n_jobs=1)

facet_lines_national_totals(result, peaks_df=peaks_df)
for k, df in result.items():
    print(f"{k:28s} rows={len(df):,}")


In [ ]:
# Chloropleth map
df = choropleth_pv_delta_gw_policy_vs_baseline(
    outputs=result
    )

In [ ]:
# Grouped bar plot with cumulative adopters
us_bars = plot_us_cum_adopters_grouped(result)

In [ ]:
result['totals'][(result['totals']['scenario'] == 'policy') & (result['totals']['year'] == 2040)]['number_of_adopters'].sum()

In [ ]:
(result['lifetime_totals'][(result['lifetime_totals']['state_abbr'] == 'NJ') & 
                           (result['lifetime_totals']['scenario'] == 'policy')]['lifetime_savings_total'].values[0]-
 result['lifetime_totals'][(result['lifetime_totals']['state_abbr'] == 'NJ') & 
                           (result['lifetime_totals']['scenario'] == 'baseline')]['lifetime_savings_total'].values[0])

In [ ]:
result['lifetime_totals'][result['lifetime_totals']['scenario'] == 'policy']['lifetime_savings_total'].sum()/result['totals'][(result['totals']['scenario'] == 'policy') & (result['totals']['year'] == 2040)]['number_of_adopters'].sum()

In [ ]:
us_policy = policy_only_bill_price_diffs_after_adoption(
    root_dir="/Volumes/Seagate Portabl/permit_power/dgen_runs/per_state_outputs/",
    year=2040,
    level="US",
    run_id=RUN_ID
)

#### Faceted comparisons by state (Baseline vs Policy)

In [ ]:
# ## State level facet plots

# # 1) Cumulative storage (kWh)
# facet_lines_by_state(
#     result["totals"], 
#     y_col="batt_kwh_cum",
#     ylabel="Cumulative Battery (kWh)",
#     title="Battery Deployment (Cumulative)"
# )

# # 2) Cumulative PV (kW)
# facet_lines_by_state(
#     result["totals"],
#     y_col="system_kw_cum",
#     ylabel="Cumulative PV (kW)",
#     title="PV Deployment (Cumulative)"
# )

# # 3) Cumulative adopters (count)
# facet_lines_by_state(
#     result["totals"],
#     y_col="number_of_adopters",
#     ylabel="Cumulative Adopters",
#     title="Adopters (Cumulative)"
# )

# # 4) Annual portfolio bill savings (cohort carry-forward)
# facet_lines_by_state(
#     result["portfolio_annual_savings"],
#     y_col="portfolio_annual_savings",
#     ylabel="Annual Portfolio\nBill Savings ($)",
#     title="Annual Portfolio Bill Savings (Carry-forward)"
# )

# # 5) Cumulative bill savings through year
# facet_lines_by_state(
#     result["cumulative_bill_savings"],
#     y_col="cumulative_bill_savings",
#     ylabel="Cumulative Bill Savings ($)",
#     title="Cumulative Bill Savings (All Cohorts)"
# )

# # # 6) Technical potential reached in 2040 (bar, sorted by Policy %)
# # bar_tech_potential_2040(result["tech_2040"])

# # 7) Median PV size by state
# facet_lines_by_state(
#     result["median_system_kw"],
#     y_col="median_system_kw",
#     ylabel="Median System Size (kW)",
#     title="Median PV System Size by Scenario",
# )

# # 8) Peak by year by state
# facet_peaks_by_state(peaks_df)

# # 8) Peak time series by state
# facet_state_peak_timeseries_from_hourly(
#     root_dir="/Volumes/Seagate Portabl/permit_power/dgen_runs/per_state_outputs",
#     run_id=RUN_ID,
#     year=2040, 
#     col_wrap=4,
#     height=2.8,
# )

# # 8) Average prices by state comparing model to EIA

# avg_prices = pd.read_csv("../../../data/average_retail_elec_price.csv")

# if not result["avg_price_2026_model"].empty:
#     try:
#         price_cmp = (result["avg_price_2026_model"]
#                      .merge(avg_prices, on="state_abbr", how="inner"))
#         price_cmp["price_per_kwh"] *= 100.0

#         # Long-form for grouped bars
#         price_melted = price_cmp.melt(
#             id_vars="state_abbr",
#             value_vars=["price_per_kwh", "cents_per_kwh"],
#             var_name="Source", value_name="Average Price (¢/kWh)"
#         ).replace({
#             "price_per_kwh": "Model",
#             "cents_per_kwh": "EIA"
#         })

#         # Sort by model price
#         order = (price_melted[price_melted["Source"] == "Model"]
#                  .sort_values("Average Price (¢/kWh)", ascending=False)
#                  ["state_abbr"].tolist())

#         plt.figure(figsize=(14,6))
#         ax = sns.barplot(
#             data=price_melted, x="state_abbr", y="Average Price (¢/kWh)",
#             hue="Source", order=order, errorbar=None
#         )
#         for container in ax.containers:
#             for bar in container:
#                 h = bar.get_height()
#                 if h > 0:
#                     ax.text(bar.get_x()+bar.get_width()/2, h-1.5, f"{h:.1f}",
#                             ha="center", va="top", color="white", fontsize=9, fontweight="bold")
#         plt.title("Average Electricity Price in 2026: Model vs EIA (Sorted by Model)")
#         plt.ylabel("¢/kWh"); plt.xlabel("State"); plt.xticks(rotation=45); plt.tight_layout(); plt.show()
#     except NameError:
#         print("avg_prices not defined; skip EIA comparison.")

In [ ]:
result['totals']['cum_new_adopters'] = result['totals'].groupby(['state_abbr', 'scenario'], as_index = False)['new_adopters'].cumsum()
df_per_household = result['totals'][(result['totals']['year'] == 2040)].merge(result['lifetime_totals'], on = ['state_abbr', 'scenario'], how = 'left') 
df_per_household_agg = df_per_household.groupby(['scenario'], as_index = False).agg({'lifetime_savings_total':'sum', 'cum_new_adopters':'sum'})
df_per_household_agg['lifetime_savings_per_household'] = df_per_household_agg['lifetime_savings_total']/df_per_household_agg['cum_new_adopters']

In [ ]:
result['portfolio_annual_savings']

In [ ]:
# NREL standard scenarios distributed PV 
nrel_dgen = nrel.merge(result['totals'][
    (result['totals']['scenario'] == 'baseline') & 
    (result['totals']['year'] == 2040)][['state_abbr', 'system_kw_cum']], on = 'state_abbr', how = 'left')

nrel_dgen['abs_error'] = nrel_dgen['nrel'] - (nrel_dgen['system_kw_cum']/1000)

fig, ax = plt.subplots(figsize=(9, 5))
ax.hist(nrel_dgen['abs_error'], bins=10, edgecolor="white", linewidth=1.0)

# Annotate counts using the drawn rectangles
heights = [p.get_height() for p in ax.patches]
for rect in ax.patches:
    h = rect.get_height()
    if h <= 0:
        continue
    x = rect.get_x() + rect.get_width() / 2
    ax.text(x, h, f"{int(round(h))}",
            ha="center", va="bottom",
            color="black", fontsize=10, fontweight="bold")

ax.set_xlabel("MW")
ax.set_ylabel("Count of States")
ax.set_title("State-level Error in MW PV Deployment \n(NREL vs. dGen)")
ax.set_ylim(0, (max(heights) * 1.12) if heights else 1)  # space for labels
ax.grid(axis="y", alpha=0.2)
plt.tight_layout()
plt.show()

In [ ]:
nrel_dgen.to_csv("../../../data/nrel_dgen_compare.csv")

In [ ]:
xlsx_path = export_compiled_results_to_excel(result, RUN_ID, peaks_df=peaks_df, include_national=True)
print("Wrote:", xlsx_path)